爬取漳州历史天气数据


In [1]:
import requests
header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
}
res_list = []
for year in range(2020,2022):
    for month in range(1,13):
        url = 'https://lishi.tianqi.com/zhangzhou/{}{:0>2d}.html'.format(year,month)
        # print(url)
        res = requests.get(url=url,headers=header)
        res_list.append(res)

In [35]:
from lxml import etree

# 2020年每天的天气情况
day = 1
csv_str = ""
for month in range(1,13):
    dom = etree.HTML(res_list[month].text)
    values = dom.xpath('/html/body/div[7]/div[1]/div[4]/ul/li')
    for one in values:
        high_temp = int(one.xpath('./div[2]/text()')[0][0:-1])
        low_temp = int(one.xpath('./div[3]/text()')[0][0:-1])
        weather = one.xpath('./div[4]/text()')[0]

        # print(day,high_temp,low_temp,weather)
        line = "{},{},{},{}\n".format(day,high_temp,low_temp,weather)
        csv_str = csv_str + line
        day += 1

head_str = "day,high,low,weather\n"
csv_str = head_str + csv_str
# print(csv_str)
with open('weather.csv','w',encoding='utf-8') as f:
    f.write(csv_str)


In [1]:
# import pandas as pd
# import numpy as np
#
# # 按照上下游顺序
# ids = [ '天宝大水港排涝站','中排渠涝站（天宝）',
#              '程溪下庄工业区水质监测点', '甘棠溪慧民花园监测点',
#         '康山溪金峰花园监测点', '芗城水利局站','中山桥水闸站', '北京路水闸站','九湖监测点','桂林排涝站','上坂']
# df = pd.read_csv('data/water/water2020.csv', usecols=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
# merge = None
# for site in ids:
#     one = df.loc[df['站点名称'] == site]
#     one.columns = ['time', 'site'] + [site + str(i) for i in range(9)]
#     one = one[['time'] + [site + str(inc)]]
#     if merge is None:
#         merge = one
#     else:
#         merge = pd.merge(merge, one, on='time')
# merge.set_index(keys='time', inplace=True)

# df = pd.read_hdf(file_name)
# df.describe()
# merge.to_hdf(file_name, key='merge', index=False)

,天宝大水港排涝站0,中排渠涝站（天宝）0,程溪下庄工业区水质监测点0,甘棠溪慧民花园监测点0,康山溪金峰花园监测点0,芗城水利局站0,中山桥水闸站0,北京路水闸站0,九湖监测点0,桂林排涝站0,上坂0
count,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000
mean,6.558555,6.322410,6.646223,6.399332,7.104915,6.218743,5.914457,6.844680,6.183382,7.011030,6.939061
std,1.160994,0.922257,0.353787,1.214922,0.902649,1.697479,1.133257,1.302965,1.933982,0.921748,0.503949
min,0.940000,0.000000,4.350000,0.000000,3.820000,0.000000,0.000000,0.000000,0.000000,0.000000,4.376000
25%,5.750000,5.700000,6.470000,6.120000,6.520000,5.020000,5.540000,6.040000,5.290000,6.690000,6.548000
50%,6.530000,6.410000,6.570000,6.610000,6.920000,6.238500,6.050000,6.780000,6.110000,6.940000,6.998000
75%,7.320000,6.980000,6.950000,7.000000,7.580000,7.530000,6.620000,7.430000,7.222500,7.330000,7.238000
max,9.560000,9.830000,7.940000,12.520000,12.670000,10.130000,9.560000,14.000000,13.800000,11.470000,9.792000


In [7]:
# print(df.head(5))
# import util
# # col_line =  df.iloc[0]
# col_line = df.iloc[:,0]
#
# util.DataLoader()


# print(col_li)

                     天宝大水港排涝站0  中排渠涝站（天宝）0  程溪下庄工业区水质监测点0  甘棠溪慧民花园监测点0  \
time                                                                     
2020-01-01 00:00:00       8.25        6.32           6.95         7.11   
2020-01-01 01:00:00       8.37        6.23           6.95         7.04   
2020-01-01 02:00:00       8.23        6.30           6.95         7.11   
2020-01-01 03:00:00       8.17        6.29           6.95         7.11   
2020-01-01 04:00:00       8.10        6.32           6.95         7.08   

                     康山溪金峰花园监测点0  芗城水利局站0  中山桥水闸站0  北京路水闸站0  九湖监测点0  桂林排涝站0  \
time                                                                          
2020-01-01 00:00:00         6.41     8.16     1.75     6.27    6.98    8.29   
2020-01-01 01:00:00         6.46     8.17     2.63     6.26    6.72    8.30   
2020-01-01 02:00:00         6.46     8.11     2.64     6.22    6.73    8.28   
2020-01-01 03:00:00         6.48     8.22     2.67     6.21    6.73    8.28   
2020-01

In [35]:
import os
import pandas as pd
import numpy as np

def generate_one_site_one_factor(factor_index,site_index, out_dir,
                                 seq_length_x=24, seq_length_y=24):
    pass
    # seq_length_x, seq_length_y = 24, 24
# pH文件
factor_index = 0
hdf_file = 'data/water/single/merge' + str(factor_index) +'.h5'
# 第一个站点
site_index = 0
site_names = "abcdefghijklmnopq"
#
out_dir = 'data/water/singlesingle/{}{}'.format(factor_index,site_names[site_index])
seq_length_x = 24
seq_length_y = 3

df = pd.read_hdf(hdf_file)
df = df.iloc[:,site_index]
x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))
y_offsets = np.sort(np.arange(1, (seq_length_y + 1), 1))

num_samples = len(df)
data = np.expand_dims(df.values, axis=-1)

x, y = [], []
min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):  # t is the index of the last observation.
    x.append(data[t + x_offsets, ...])
    y.append(data[t + y_offsets, ...])
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)

x = np.squeeze(x)
y = np.squeeze(y)

print("x shape: ", x.shape, ", y shape: ", y.shape)
# Write the data into npz file.
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
x_test, y_test = x[-num_test:], y[-num_test:]

exi =  os.path.exists(out_dir)
if not exi:
    os.mkdir(out_dir)

for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)

    np.savez_compressed(
        os.path.join(out_dir, f"{cat}.npz"),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )

x shape:  (8758, 24) , y shape:  (8758, 3)
train x:  (6131, 24) y: (6131, 3)
val x:  (875, 24) y: (875, 3)
test x:  (1752, 24) y: (1752, 3)


In [1]:
import util
dataloader = util.load_dataset('data/water/singlesingle/0a',64,64,64,False)


# 构建模型
import torch
import torch.nn as nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

##### LSTM
import utils.better_LSTM as better
model = better.LSTM(24,3).to(device)

##### MLP
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(24, 64),
#             nn.ReLU(),
#             nn.Linear(64, 64),
#             nn.ReLU(),
#             nn.Linear(64, 3)
#         )
# 
#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits
# 
# model = NeuralNetwork().to(device)


model = model.double()
print(model)

Using cpu device
LSTM(
  24, 3, batch_first=True
  (input_drop): VariationalDropout()
  (output_drop): VariationalDropout()
)


In [2]:
# loss_fn = nn.CrossEntropyLoss()
loss_fn = util.masked_mae
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    # size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader['train_loader'].get_iterator()):
        X = np.expand_dims(X,axis=1)
        y = np.expand_dims(y,axis=1)
        X = torch.tensor(X).to(device)
        y = torch.tensor(y).to(device)
        pred = model(X)
        loss = loss_fn(pred[0], y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



import numpy as np
def test(dataloader, model, loss_fn):
    model.eval()
    loss_list = []
    with torch.no_grad():
        for X, y in dataloader['test_loader'].get_iterator():
            X = np.expand_dims(X,axis=1)
            y = np.expand_dims(y,axis=1)

            X = torch.tensor(X).to(device)
            y = torch.tensor(y).to(device)

            pred = model(X)
            loss_list.append(loss_fn(pred[0], y).item())

    loss_result = np.mean(loss_list)
    return loss_result

In [3]:
epochs = 100
for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss_fn, optimizer)
    loss_result = test(dataloader, model, loss_fn)
    print("Epoch:{},loss:{}".format(t+1,loss_result))
print("Done!")

Epoch:1,loss:6.164609377554983
Epoch:2,loss:6.154703157483261
Epoch:3,loss:6.150853947590463
Epoch:4,loss:6.148724537239963
Epoch:5,loss:6.14731971082741
Epoch:6,loss:6.14627814780647
Epoch:7,loss:6.145430155527878
Epoch:8,loss:6.144676859829291
Epoch:9,loss:6.143943897260079
Epoch:10,loss:6.14315345809176
Epoch:11,loss:6.142189396115213
Epoch:12,loss:6.140813307982882
Epoch:13,loss:6.138350625485025
Epoch:14,loss:6.13168857570564
Epoch:15,loss:6.069121620683194
Epoch:16,loss:5.915023040683985
Epoch:17,loss:5.907125724872873
Epoch:18,loss:5.904173404611043
Epoch:19,loss:5.902505395163085
Epoch:20,loss:5.901383222014642
Epoch:21,loss:5.90055159662689
Epoch:22,loss:5.899896600253608
Epoch:23,loss:5.899358902173646
Epoch:24,loss:5.898904182626071
Epoch:25,loss:5.898511013504766
Epoch:26,loss:5.898165212648844
Epoch:27,loss:5.89785695102893
Epoch:28,loss:5.89757915699108
Epoch:29,loss:5.897326582530264
Epoch:30,loss:5.89709522992768
Epoch:31,loss:5.896881985208731
Epoch:32,loss:5.896684375